In [1]:
import glob
import pandas as pd

rawData = []
outfile = "results.pkl"

In [5]:
for file in glob.glob("*/*.log"):
    print(file)
    workloadParts = (file.split("/")[1]).split("_")
    platform = workloadParts[0]
    functionType = (workloadParts[1])
    regression = int(workloadParts[2])
    #    with open(file) as logfile:
    with open(file) as logfile:
        for line in logfile:
            # for DuetFaaS
            if line.__contains__("isColdStart: true"):
                print("skipping due to cold start")
                continue
            if line.__contains__("Response Time"):
                
                f1Durations = []
                f2Durations = []
                line = line.strip()
                parts = line.split(" ")
                if "." in parts[4]:
                    f1 = int(parts[4].split(".")[0])
                else:
                    f1 = int(parts[4][0:parts[4].index("m")])
                if "." in parts[20]:
                    f2 = int(parts[20].split(".")[0])
                else:
                    f2 = int(parts[20][0:parts[20].index("m")])
                #print(f" f1 is {f1}, f2 is {f2}")
                f1Durations.append(f1)
                f2Durations.append(f2)
                row = {}
                row["platform"] = "aws"
                row["functionType"] = functionType
                row["regression"] = regression
                row["mode"] = "duet"
                #print(f"platform is {platform}, functionType is {functionType}, regression is {regression}")
                for i in range(0, len(f1Durations)):
                    row["f1"] = f1Durations[i]
                    row["f2"] = f2Durations[i]
                    row["change"] = f2Durations[i] / f1Durations[i]
                    rawData.append(row)
            # for faasterBench
            if line.__contains__("faaster"):
                if(file.__contains__("aws")):
                    line = line[:-4]
                    if line.__contains__("modeA") and not (line.__contains__("cold start")):
                        line = line.strip()
                        parts2 = line.split(" ")
                        #print(line)
                        row = {}
                        row["platform"] = platform
                        row["functionType"] = functionType
                        row["regression"] = regression
                        row["mode"] = "modeA"

                        f1Durations = []
                        f2Durations = []
                        f2Found = False

                        for i in range(4,len(parts2)):
                            if parts2[i] == "f2":
                                break
                            
                            #if int(parts2[i]) < 2000 and int(parts2[i+4]) < 2100:
                            f1Durations.append(int(parts2[i]))
                            f2Durations.append(int(parts2[i+4]))
                        for i in range(0, len(f1Durations)):
                            #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                            row["f1"] = f1Durations[i]
                            row["f2"] = f2Durations[i]
                            row["change"] = f2Durations[i] / f1Durations[i]
                            rawData.append(row)
                    if line.__contains__("modeB") and not (line.__contains__("cold start")):
                        line = line.strip()
                        parts2 = line.split(" ")
                        #print(line)
                        row = {}
                        row["platform"] = platform
                        row["functionType"] = functionType
                        row["regression"] = regression
                        row["mode"] = "modeB"
                        f1Durations = []

                        for i in range(4,len(parts2)):
                            if parts2[i] == "f2":
                                break
                            else:
                                if int(parts2[i]) < 2000:
                                    f1Durations.append(int(parts2[i]))

                        for i in range(0, len(f1Durations)):
                            #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                            row["f1"] = f1Durations[i]
                            row["f2"] = 0
                            row["change"] = 0
                            rawData.append(row)

                    if line.__contains__("modeC") and not (line.__contains__("cold start")):
                        line = line.strip()
                        parts2 = line.split(" ")
                        #print(line)
                        row = {}
                        row["platform"] = platform
                        row["functionType"] = functionType
                        row["regression"] = regression
                        row["mode"] = "modeC"
                        f2Durations = []

                        for i in range(5,len(parts2)):
                            if parts2[i] == "\",":
                                break
                            else:
                                if int(parts2[i]) < 2100:
                                    f2Durations.append(int(parts2[i]))

                        for i in range(0, len(f2Durations)):
                            #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                            row["f1"] = 0
                            row["f2"] = f2Durations[i]
                            row["change"] = 0
                            rawData.append(row)
                if(file.__contains__("google")):
                    if line.__contains__("textPayload"):
                        if line.__contains__("modeA") and not line.__contains__("cold start"):
                            line = line.strip()
                            parts2 = line.split(" ")
                            #print(line)
                            row = {}
                            row["platform"] = platform
                            row["functionType"] = functionType
                            row["regression"] = regression
                            row["mode"] = "modeA"

                            f1Durations = []
                            f2Durations = []
                            f2Found = False

                            for i in range(4,len(parts2)):
                                if parts2[i] == "f2":
                                    break
                                
                                else:
                                    if int(parts2[i]) < 1800 and int(parts2[i+4]) < 1800:
                                        f1Durations.append(int(parts2[i]))
                                        f2Durations.append(int(parts2[i+4]))

                            for i in range(0, len(f1Durations)):
                                #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                                row["f1"] = f1Durations[i]
                                row["f2"] = f2Durations[i]
                                row["change"] = f2Durations[i] / f1Durations[i]
                                rawData.append(row)
                        if line.__contains__("modeB") and not line.__contains__("cold start"):
                            line = line.strip()
                            parts2 = line.split(" ")
                            #print(line)
                            row = {}
                            row["platform"] = platform
                            row["functionType"] = functionType
                            row["regression"] = regression
                            row["mode"] = "modeB"

                            f1Durations = []

                            for i in range(4,len(parts2)):
                                if parts2[i] == "f2":
                                    break
                                else:
                                    if int(parts2[i]) < 1800:
                                        f1Durations.append(int(parts2[i]))

                            for i in range(0, len(f1Durations)):
                                #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                                row["f1"] = f1Durations[i]
                                row["f2"] = 0
                                row["change"] = 0
                                rawData.append(row)

                        if line.__contains__("modeC") and not line.__contains__("cold start"):
                            line = line.strip()
                            parts2 = line.split(" ")
                            #print(line)
                            row = {}
                            row["platform"] = platform
                            row["functionType"] = functionType
                            row["regression"] = regression
                            row["mode"] = "modeC"

                            f2Durations = []

                            for i in range(5,len(parts2)):
                                if parts2[i] == "\",":
                                    break
                                else:
                                    if int(parts2[i]) < 1800:
                                        f2Durations.append(int(parts2[i]))

                            for i in range(0, len(f2Durations)):
                                #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                                row["f1"] = 0
                                row["f2"] = f2Durations[i]
                                row["change"] = 0
                                rawData.append(row)



duet/duet_mem_0_07_11_11.log
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
duet/duet_cpu_5_07_11_11.log
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
duet/duet_cpu_0_07_11_11.log
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
duet/duet_mem_5_07_11_11.log
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
aws/aws_cpu_0_07_10_09.log
aws/aws_mem_5_07_10_09.log
aws/aws_mem_0_07_10_09.log
aws/aws_cpu_5_07_10_09.log


In [6]:
df_results = pd.DataFrame(rawData)
df_results.head()
df_results.describe()
print(df_results.loc[(df_results["mode"] == "duet")&
                     (df_results["functionType"] == "mem")&
                     (df_results["regression"] == 5)])
df_results.to_pickle(outfile)
#df_results.to_csv("nils2.csv")

     platform functionType  regression  mode  f1  f2    change
4481      aws          mem           5  duet  68  70  1.029412
4482      aws          mem           5  duet  61  66  1.081967
4483      aws          mem           5  duet  69  68  0.985507
4484      aws          mem           5  duet  65  71  1.092308
4485      aws          mem           5  duet  52  53  1.019231
...       ...          ...         ...   ...  ..  ..       ...
5970      aws          mem           5  duet  48  50  1.041667
5971      aws          mem           5  duet  47  51  1.085106
5972      aws          mem           5  duet  49  50  1.020408
5973      aws          mem           5  duet  47  50  1.063830
5974      aws          mem           5  duet  48  50  1.041667

[1494 rows x 7 columns]
